In [ ]:
import os
import re
import torch
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/TextClassificationAttention
#!kaggle datasets download adityajn105/glove6b50d
#!kaggle datasets download dushyantv/consumer_complaints

/content/drive/MyDrive/TextClassificationAttention


In [ ]:
#!unzip consumer_complaints.zip
#!unzip glove6b50d.zip

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/TextClassificationAttention/Consumer_Complaints.csv")
print(data.columns)
print(data.head())


Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer Complaint', 'Company Public Response', 'Company', 'State',
       'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via',
       'Date Sent to Company', 'Company Response to Consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID',
       'Unnamed: 18'],
      dtype='object')
  Date received           Product     Sub-product  \
0    03-12-2014          Mortgage  Other mortgage   
1    10-01-2016  Credit reporting             NaN   
2    10/17/2016     Consumer Loan    Vehicle loan   
3    06-08-2014       Credit card             NaN   
4    09/13/2014   Debt collection     Credit card   

                                      Issue                   Sub-issue  \
0  Loan modification,collection,foreclosure                         NaN   
1    Incorrect information on credit report              Account status   
2                Managing the loan or lease                  

# Pre processing

In [ ]:
lr = 0.0005
vec_len = 50
seq_len = 20
num_epochs = 50
label_col = "Product"
tokens_path = "/content/drive/MyDrive/TextClassificationAttention/tokens.pkl"
labels_path = "/content/drive/MyDrive/TextClassificationAttention/labels.pkl"
data_path = "/content/drive/MyDrive/TextClassificationAttention/Consumer_Complaints.csv"
model_path = "/content/drive/MyDrive/TextClassificationAttention/attention.pth"
vocabulary_path = "/content/drive/MyDrive/TextClassificationAttention/vocabulary.pkl"
embeddings_path = "/content/drive/MyDrive/TextClassificationAttention/embeddings.pkl"
glove_vector_path = "/content/drive/MyDrive/TextClassificationAttention/glove.6B.50d.txt"
text_col_name = "Consumer Complaint"
label_encoder_path = "/content/drive/MyDrive/TextClassificationAttention/label_encoder.pkl"
product_map = {'Vehicle loan or lease': 'vehicle_loan',
               'Credit reporting, credit repair services, or other personal consumer reports': 'credit_report',
               'Credit card or prepaid card': 'card',
               'Money transfer, virtual currency, or money service': 'money_transfer',
               'virtual currency': 'money_transfer',
               'Mortgage': 'mortgage',
               'Payday loan, title loan, or personal loan': 'loan',
               'Debt collection': 'debt_collection',
               'Checking or savings account': 'savings_account',
               'Credit card': 'card',
               'Bank account or service': 'savings_account',
               'Credit reporting': 'credit_report',
               'Prepaid card': 'card',
               'Payday loan': 'loan',
               'Other financial service': 'others',
               'Virtual currency': 'money_transfer',
               'Student loan': 'loan',
               'Consumer Loan': 'loan',
               'Money transfers': 'money_transfer'}

In [ ]:
def save_file(name, obj):
    """
    Function to save an object as pickle file
    """
    with open(name, 'wb') as f:
        pickle.dump(obj, f)


def load_file(name):
    """
    Function to load a pickle object
    """
    return pickle.load(open(name, "rb"))

Glove embedding

In [ ]:
with open(glove_vector_path, "rt") as f:
    emb = f.readlines()

In [ ]:
vocabulary, embeddings = [], []

for item in emb:
    vocabulary.append(item.split()[0])
    embeddings.append(item.split()[1:])

In [ ]:
embeddings = np.array(embeddings, dtype=np.float32)

In [ ]:
vocabulary = ["<pad>", "<unk>"] + vocabulary

In [ ]:
embeddings = np.vstack([np.ones(50, dtype=np.float32),
                        np.mean(embeddings, axis=0),
                        embeddings])

In [ ]:
save_file(embeddings_path, embeddings)
save_file(vocabulary_path, vocabulary)

# Process text

In [ ]:
data = pd.read_csv(data_path)

In [ ]:
data.dropna(subset=[text_col_name], inplace=True)

In [ ]:
data.replace({label_col: product_map}, inplace=True)

# Encode labels

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(data[label_col])
labels = label_encoder.transform(data[label_col])

In [ ]:
save_file(labels_path, labels)
save_file(label_encoder_path, label_encoder)

# Process text column


In [ ]:
input_text = list(data[text_col_name])

In [ ]:
len(input_text)

277814

# Convert to lowecase

In [ ]:
input_text = [i.lower() for i in tqdm(input_text)]

100%|██████████| 277814/277814 [00:00<00:00, 737053.34it/s]


# Remove punctuations except apostrophe

In [ ]:
input_text = [re.sub(r"[^\w\d'\s]+", " ", i)
              for i in tqdm(input_text)]

100%|██████████| 277814/277814 [00:13<00:00, 21047.88it/s]


# remove Numbers


In [ ]:
input_text = [re.sub("\d+", "", i) for i in tqdm(input_text)]

100%|██████████| 277814/277814 [00:08<00:00, 31598.33it/s]


# Remove more than one consecutive instance of 'x'

In [ ]:
input_text = [re.sub(r'[x]{2,}', "", i) for i in tqdm(input_text)]

100%|██████████| 277814/277814 [00:05<00:00, 47203.01it/s]


# Remove multiple spaces with single space

In [ ]:
nput_text = [re.sub(' +', ' ', i) for i in tqdm(input_text)]

100%|██████████| 277814/277814 [00:22<00:00, 12217.23it/s]


# Tokenize the text

In [ ]:
import nltk
nltk.download('punkt_tab')

#tokens = [word_tokenize(t) for t in tqdm(input_text, mininterval=60)]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Take the first 20 tokens in each complaint text

In [ ]:
tokens = [i[:20] if len(i) > 19 else ['<pad>'] * (20 - len(i)) + i
          for i in tqdm(tokens)]


100%|██████████| 277814/277814 [00:03<00:00, 73475.70it/s] 


# Convert tokens to integer indices from vocabulary

In [ ]:
def token_index(tokens, vocabulary, missing='<unk>'):
    """
    :param tokens: List of word tokens
    :param vocabulary: All words in the embeddings
    :param missing: Token for words not present in the vocabulary
    :return: List of integers representing the word tokens
    """
    idx_token = []
    for text in tqdm(tokens):
        idx_text = []
        for token in text:
            if token in vocabulary:
                idx_text.append(vocabulary.index(token))
            else:
                idx_text.append(vocabulary.index(missing))
        idx_token.append(idx_text)
    return idx_token


# save the tokens

In [ ]:
tokens = token_index(tokens, vocabulary)
save_file(tokens_path, tokens)

100%|██████████| 277814/277814 [1:01:17<00:00, 75.54it/s]


#Create attention

In [ ]:
class AttentionModel(nn.Module):

    def __init__(self, vec_len, seq_len, n_classes):
        super(AttentionModel, self).__init__()
        self.vec_len = vec_len
        self.seq_len = seq_len
        self.attn_weights = torch.cat([torch.tensor([[0.]]),
                                       torch.randn(vec_len, 1) /
                                       torch.sqrt(torch.tensor(vec_len))])
        self.attn_weights.requires_grad = True
        self.attn_weights = nn.Parameter(self.attn_weights)
        self.activation = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)
        self.linear = nn.Linear(vec_len + 1, n_classes)

    def forward(self, input_data):
        hidden = torch.matmul(input_data, self.attn_weights)
        hidden = self.activation(hidden)
        attn = self.softmax(hidden)
        attn = attn.repeat(1, 1, self.vec_len + 1).reshape(attn.shape[0],
                                                           self.seq_len,
                                                           self.vec_len + 1)
        attn_output = input_data * attn
        attn_output = torch.sum(attn_output, axis=1)
        output = self.linear(attn_output)
        return output

#Create PyTorch dataset

In [ ]:
class TextDataset(torch.utils.data.Dataset):

    def __init__(self, tokens, embeddings, labels):
        """
        :param tokens: List of word tokens
        :param embeddings: Word embeddings (from glove)
        :param labels: List of labels
        """
        self.tokens = tokens
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        emb = torch.tensor(self.embeddings[self.tokens[idx], :])
        input_ = torch.cat((torch.ones(emb.shape[0],1), emb), dim=1)
        return torch.tensor(self.labels[idx]), input_

#Function to train the model

In [ ]:
def train(train_loader, valid_loader, model, criterion, optimizer,
          device, num_epochs, model_path):
    """
    Function to train the model
    :param train_loader: Data loader for train dataset
    :param valid_loader: Data loader for validation dataset
    :param model: Model object
    :param criterion: Loss function
    :param optimizer: Optimizer
    :param device: CUDA or CPU
    :param num_epochs: Number of epochs
    :param model_path: Path to save the model
    """
    best_loss = 1e8
    for i in range(num_epochs):
        print(f"Epoch {i+1} of {num_epochs}")
        valid_loss, train_loss = [], []
        model.train()
        # Train loop
        for batch_labels, batch_data in tqdm(train_loader):
            # Move data to GPU if available
            batch_labels = batch_labels.to(device)
            batch_data = batch_data.to(device)
            # Forward pass
            batch_output = model(batch_data)
            batch_output = torch.squeeze(batch_output)
            # Calculate loss
            loss = criterion(batch_output, batch_labels)
            train_loss.append(loss.item())
            optimizer.zero_grad()
            # Backward pass
            loss.backward()
            # Gradient update step
            optimizer.step()
        model.eval()
        # Validation loop
        for batch_labels, batch_data in tqdm(valid_loader):
            # Move data to GPU if available
            batch_labels = batch_labels.to(device)
            batch_data = batch_data.to(device)
            # Forward pass
            batch_output = model(batch_data)
            batch_output = torch.squeeze(batch_output)
            # Calculate loss
            loss = criterion(batch_output, batch_labels)
            valid_loss.append(loss.item())
        t_loss = np.mean(train_loss)
        v_loss = np.mean(valid_loss)
        print(f"Train Loss: {t_loss}, Validation Loss: {v_loss}")
        if v_loss < best_loss:
            best_loss = v_loss
            # Save model if validation loss improves
            torch.save(model.state_dict(), model_path)
        print(f"Best Validation Loss: {best_loss}")

#Function to test the model

In [ ]:
def test(test_loader, model, criterion, device):
    """
    Function to test the model
    :param test_loader: Data loader for test dataset
    :param model: Model object
    :param criterion: Loss function
    :param device: CUDA or CPU
    """
    model.eval()
    test_loss = []
    test_accu = []
    for batch_labels, batch_data in tqdm(test_loader):
        # Move data to device
        batch_labels = batch_labels.to(device)
        batch_data = batch_data.to(device)
        # Forward pass
        batch_output = model(batch_data)
        batch_output = torch.squeeze(batch_output)
        # Calculate loss
        loss = criterion(batch_output, batch_labels)
        test_loss.append(loss.item())
        batch_preds = torch.argmax(batch_output, axis=1)
        # Move predictions to CPU
        if torch.cuda.is_available():
            batch_labels = batch_labels.cpu()
            batch_preds = batch_preds.cpu()
        # Compute accuracy
        test_accu.append(accuracy_score(batch_labels.detach().
                                        numpy(),
                                        batch_preds.detach().
                                        numpy()))
    test_loss = np.mean(test_loss)
    test_accu = np.mean(test_accu)
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accu}")

#Train attention model

In [ ]:
tokens = load_file(tokens_path)
labels = load_file(labels_path)
embeddings = load_file(embeddings_path)
label_encoder = load_file(label_encoder_path)
num_classes = len(label_encoder.classes_)
vocabulary = load_file(vocabulary_path)

Split data into train, validation and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tokens, labels,
                                                    test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                      y_train,
                                                      test_size=0.25)

Create PyTorch datasets

In [ ]:
train_dataset = TextDataset(X_train, embeddings, y_train)
valid_dataset = TextDataset(X_valid, embeddings, y_valid)
test_dataset = TextDataset(X_test, embeddings, y_test)

Create data loaders

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=16,
                                           shuffle=True,
                                           drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=16)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=16)

Create model object

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available()
                      else "cpu")
model = AttentionModel(vec_len, seq_len, num_classes)

Move the model to GPU if available


In [ ]:
if torch.cuda.is_available():
    model = model.cuda()

Define loss function and optimizer

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Training loop

In [ ]:
train(train_loader, valid_loader, model, criterion, optimizer,
      device, num_epochs, model_path)

Epoch 1 of 50


100%|██████████| 3473/3473 [00:04<00:00, 841.76it/s]


Train Loss: 1.255738702538856, Validation Loss: 1.1066387673379225
Best Validation Loss: 1.1066387673379225
Epoch 2 of 50


100%|██████████| 3473/3473 [00:04<00:00, 824.02it/s]


Train Loss: 1.0755504596385554, Validation Loss: 1.056090299244954
Best Validation Loss: 1.056090299244954
Epoch 3 of 50


...

Train Loss: 0.9791590079999881, Validation Loss: 0.9805058835306533
Best Validation Loss: 0.9801879286628754
Epoch 49 of 50


100%|██████████| 3473/3473 [00:04<00:00, 790.36it/s]


Train Loss: 0.9791298950429634, Validation Loss: 0.9797732829790173
Best Validation Loss: 0.9797732829790173
Epoch 50 of 50


100%|██████████| 3473/3473 [00:05<00:00, 686.16it/s]


Train Loss: 0.9788074452300565, Validation Loss: 0.9800696365262808
Best Validation Loss: 0.9797732829790173


Test the model

In [ ]:
test(test_loader, model, criterion, device)

100%|██████████| 3473/3473 [00:07<00:00, 495.37it/s]

Test Loss: 0.9828092007928096, Test Accuracy: 0.6694434992016335


#Predict on new text

In [ ]:
input_text = '''I am a victim of Identity Theft & currently have an Experian account that
I can view my Experian Credit Report and getting notified when there is activity on
my Experian Credit Report. For the past 3 days I've spent a total of approximately 9
hours on the phone with Experian. Every time I call I get transferred repeatedly and
then my last transfer and automated message states to press 1 and leave a message and
someone would call me. Every time I press 1 I get an automatic message stating than you
before I even leave a message and get disconnected. I call Experian again, explain what
is happening and the process begins again with the same end result. I was trying to have
this issue attended and resolved informally but I give up after 9 hours. There are hard
hit inquiries on my Experian Credit Report that are fraud, I didn't authorize, or recall
and I respectfully request that Experian remove the hard hit inquiries immediately just
like they've done in the past when I was able to speak to a live Experian representative
in the United States. The following are the hard hit inquiries : BK OF XXXX XX/XX/XXXX
XXXX XXXX XXXX  XX/XX/XXXX XXXX  XXXX XXXX  XX/XX/XXXX XXXX  XX/XX/XXXX XXXX  XXXX
XX/XX/XXXX'''

Process input text

In [ ]:
input_text = input_text.lower()
input_text = re.sub(r"[^\w\d'\s]+", " ", input_text)
input_text = re.sub("\d+", "", input_text)
input_text = re.sub(r'[x]{2,}', "", input_text)
input_text = re.sub(' +', ' ', input_text)
tokens = word_tokenize(input_text)

In [ ]:
tokens = ['<pad>']*(20-len(tokens))+tokens

In [ ]:
idx_token = []
for token in tokens:
    if token in vocabulary:
        idx_token.append(vocabulary.index(token))
    else:
        idx_token.append(vocabulary.index('<unk>'))

In [ ]:
token_emb = embeddings[idx_token,:]
token_emb = token_emb[:seq_len, :]
inp = torch.from_numpy(token_emb)

In [ ]:
inp = torch.cat((torch.ones(inp.shape[0],1), inp), dim=1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available()
                      else "cpu")

In [ ]:
inp = inp.to(device)
inp = torch.unsqueeze(inp, 0)

In [ ]:
label_encoder = load_file(label_encoder_path)
num_classes = len(label_encoder.classes_)

In [ ]:
# Create model object
model = AttentionModel(vec_len, seq_len, num_classes)

# Load trained weights
model.load_state_dict(torch.load(model_path))

# Move the model to GPU if available
if torch.cuda.is_available():
    model = model.cuda()

# Forward pass
out = torch.squeeze(model(inp))

# Find predicted class
prediction = label_encoder.classes_[torch.argmax(out)]
print(f"Predicted  Class: {prediction}")

Predicted  Class: credit_report


<ipython-input-34-c6eeda1fc3f5>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


#save model

In [ ]:
import torch

def save_model(model, model_path):
    """
    Saves the model to a file.

    Args:
        model: The model to save.
        model_path: The path to save the model to.
    """
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

def load_model(model, model_path):
    """
    Loads the model from a file.

    Args:
        model: The model to load.
        model_path: The path to load the model from.
    """
    model.load_state_dict(torch.load(model_path))
    print(f"Model loaded from {model_path}")

In [ ]:
model_path = "/content/drive/MyDrive/TextClassificationAttention/models"
# Create an instance of the model
model_instance = AttentionModel(vec_len, seq_len, num_classes)
save_model(model_instance, model_path)  # Save the model instance

Model saved to /content/drive/MyDrive/TextClassificationAttention/models


In [ ]:
model = AttentionModel(vec_len, seq_len, num_classes)  # Create the model instance
load_model(model, model_path)  # Load the saved weights

Model loaded from /content/drive/MyDrive/TextClassificationAttention/models


<ipython-input-35-ff72fdc4ab4f>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
